### Libraries

In [51]:
%%capture
%reset -f                        # clear all variables from the workspace
'generic imports'
import os
import pandas as pd
import sys
sys.path.append(os.path.abspath('..'))
from src import utils   
import datetime        

'machine learning imports'
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.pretraining import TabNetPretrainer
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

### GPU

In [52]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(DEVICE))

Using cpu


### Load data

In [53]:
data_dir = os.path.abspath('../data')

# Non-augmented dataset
df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k.csv'), low_memory=False)
AUGMENTATION = 'None'

# SMOTE augmented dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_SMOTE.csv'), low_memory=False)
# AUGMENTATION = 'SMOTE'

# SMOTE-NC augmented dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_SMOTE_NC.csv'), low_memory=False)
# AUGMENTATION = 'SMOTE-NC'

# RealTabFormer augmentation dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_RealTabFormer.csv'), low_memory=False)
# AUGMENTATION = 'RealTabFormer'

# GReaT augmentation dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_GReaT.csv'), low_memory=False)
# AUGMENTATION = 'GReaT'


# Test data for all datasets
df_test = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_test.csv'), low_memory=False)

### Data preparation

In [54]:
# drop columns mbtcp.unit_id and mbtcp.trans_id from train and test data    
df_train = df_train.drop(['mbtcp.unit_id', 'mbtcp.trans_id'], axis=1)
df_test = df_test.drop(['mbtcp.unit_id', 'mbtcp.trans_id'], axis=1)

# extract features from df_train
features = [col for col in df_train.columns if col not in ["Attack_label"]+["Attack_type"]] 

# index of categorical features in df_train
cat_idxs = [i for i, f in enumerate(features) if f in df_train.select_dtypes(include="object").columns]

# number of unique values in each categorical column
cat_dims = [len(df_train[f].unique()) for i, f in enumerate(features) if f in df_train.select_dtypes(include="object").columns]

# AQUI Categorical columns in df_train
categorical_columns = [f for f in features if f in df_train.select_dtypes(include="object").columns]

X_train_encoded = pd.get_dummies(X_train, columns=categorical_columns)


# converts X_train and y_train to numpy arrays
X_train = df_train[features]
y_train = df_train["Attack_type"]

# converts X_test and y_test to numpy arrays
X_test = df_test[features]
y_test = df_test["Attack_type"]

# size of X_train, y_train  X_test, y_test
print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}, y_test shape: {y_test.shape}')
print(cat_idxs)

X_train shape: (536515, 44), y_train shape: (536515,)
X_test shape: (381934, 44), y_test shape: (381934,)
[6, 7, 8, 25, 31, 39, 40]


In [55]:
X_train

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.request.method,http.referer,http.request.version,http.response,...,mqtt.hdrflags,mqtt.len,mqtt.msg_decoded_as,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
536511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
536512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
536513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0


#### Label encoding

In [56]:
# instantiate the label encoder
le = LabelEncoder()

# fit and encode the training labels
y_train = le.fit_transform(y_train)

# encode the test labels
y_test = le.transform(y_test)

print('Attack_type and encoded labels:\n')
for i, label in enumerate(le.classes_):
    print(f'{label:23s} {i:d}')

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


### Model Training

In [57]:
# # embedding dimension for each categorical column
# cat_emb_dim = 10 

# # initialize embedder 
# cat_embedder = TabNetPretrainer(cat_dims, cat_emb_dim, cat_idxs)

# # instantiate TabNetClassifier model
# tabnet = TabNetClassifier(device_name = DEVICE,
#                           cat_idxs=cat_idxs,
#                           cat_dims=cat_dims,
#                           cat_emb_dim=cat_emb_dim)

In [58]:
# from X_train extract indices of categorical columns
cat_idxs = [i for i, f in enumerate(features) if f in X_train.select_dtypes(include="object").columns]
cat_idxs

[6, 7, 8, 25, 31, 39, 40]

In [42]:
tabnet = TabNetClassifier(
    n_d=64, n_a=64, n_steps=5,
    gamma=1.5, n_independent=2, n_shared=2,
    cat_idxs=cat_idxs,
    cat_dims=cat_dims,
    cat_emb_dim=10,
    lambda_sparse=1e-4, momentum=0.3, clip_value=2.,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params = {"gamma": 0.95, "step_size": 20},
    scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15
)

c:\ambientes_virtuais_py\data_augment\data_augment\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [46]:
tabnet.fit(X_train=X_train.values, y_train=y_train,
           augmentations=None,
           max_epochs=100, 
           patience=10,
           batch_size=1024,
           virtual_batch_size=128
           )

ValueError: could not convert string to float: 'GET'

In [45]:
y_train

array([4, 8, 4, ..., 7, 7, 7])

In [29]:
X_train

array([[0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ...,
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0]], dtype=object)

In [ ]:
clf = TabNetClassifier(
    n_d=64, n_a=64, n_steps=5,
    gamma=1.5, n_independent=2, n_shared=2,
    cat_idxs=cat_idxs,
    cat_dims=cat_dims,
    cat_emb_dim=cat_emb_dim,
    lambda_sparse=1e-4, momentum=0.3, clip_value=2.,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params = {"gamma": 0.95, "step_size": 20},
    scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15
)

max_epochs = 100 if not os.getenv("CI", False) else 2

clf.fit(
    X_train=X_train, y_train=y_train,
    max_epochs=max_epochs, patience=100,
    batch_size=16384, virtual_batch_size=256
)

#### Save model

In [ ]:
saved_filename = tabnet.save_model('checkpoints/tabnet')

### Model Evaluation

In [ ]:
predictions = tabnet.predict(X_test)

#### Metrics

In [ ]:
# Calculare and printe a nice board with precision, Recall, F1-score, AUC, Accuracy without classificaiton report
print("Model Evaluation Metrics")
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Accuracy: {}".format(metrics.accuracy_score(y_test, predictions)))
print("Precision: {}".format(metrics.precision_score(y_test, predictions, average='weighted')))
print("Recall: {}".format(metrics.recall_score(y_test, predictions, average='weighted')))
print("F1: {}".format(metrics.f1_score(y_test, predictions, average='weighted')))
print("AUC: {}".format(metrics.roc_auc_score(y_test, predictions, average='weighted')))
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print("\n")

#### Confusion matrix

In [ ]:
conf_mat = metrics.confusion_matrix(y_test, predictions)
# Attack_type: ['DDoS_UDP' 'Password' 'DDoS_TCP' 'Backdoor' 'DDoS_ICMP' 'Port_Scanning'
#  'Vulnerability_scanner' 'SQL_injection' 'DDoS_HTTP' 'Uploading' 'XSS'
#  'Ransomware' 'MITM' 'Fingerprinting' 'Normal']

# conf_mat_df = pd.DataFrame(conf_mat, index = ['DDoS_UDP', 'Password', 'DDoS_TCP', 'Backdoor', 'DDoS_ICMP', 'Port_Scanning', 'Vulnerability_scanner', 'SQL_injection', 'DDoS_HTTP', 'Uploading', 'XSS', 'Ransomware', 'MITM', 'Fingerprinting', 'Normal'], columns = ['DDoS_UDP', 'Password', 'DDoS_TCP', 'Backdoor', 'DDoS_ICMP', 'Port_Scanning', 'Vulnerability_scanner', 'SQL_injection', 'DDoS_HTTP', 'Uploading', 'XSS', 'Ransomware', 'MITM', 'Fingerprinting', 'Normal'])

conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'
print(conf_mat_df)

### Save Results

In [ ]:
# create dictionary for results
results = {
    "model": "TabNet",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": metrics.accuracy_score(y_test, predictions),
    "precision": metrics.precision_score(y_test, predictions, average='weighted'),
    "recall": metrics.recall_score(y_test, predictions, average='weighted'),
    "f1": metrics.f1_score(y_test, predictions, average='weighted'),
    "auc": metrics.roc_auc_score(y_test, predictions, average='weighted')
    }

# save results to csv   
utils.save_results([results], 'results/TabNet.csv')